## 2026.01.25(Sun)_Synthetic promoter libarary
### CpG iland　archtecture containing library

In [2]:
"""
統合人工プロモータライブラリ生成システム v4.0
=============================================

【主要な改良点 v4.0】
1. Constitutive強発現型TF ~60種類（JASPAR/HT-SELEX基準）
2. TFBMモジュール内部spacer: 5bp / 10bp の2バージョン
3. CpG-like (GC-rich Dispersed TSS) カテゴリ追加
4. 既存TATA-less設計の維持
5. 全spacerはGC含有率40-60%、二次構造回避設計

【構造】
[TFBM Module] - [Connector Spacer: 4/7/10 bp] - [Core Promoter: 34種類] - [Barcode: 20 bp]

【ライブラリサイズ】
総数 = TF数(~60) × 内部spacer(2) × コピー数(2) × connector spacer(3) × core promoter(34)
     = 58 × 2 × 2 × 3 × 34 = 23,664 プロモータ

References:
-----------
- JASPAR 2024: Rauluseviciute et al. (2024) Nucleic Acids Res
- HT-SELEX: Jolma et al. (2013) Cell 152:327-339
- CpG islands: Deaton & Bird (2011) Genes Dev 25:1010-1022
- Core promoters: Haberle & Stark (2018) Nat Rev Mol Cell Biol 19:621-637
"""

import pandas as pd
import numpy as np

#型ヒントを使うために,typing moduleを入れる。
from typing import Dict, List, Tuple, Optional, Set

#データを格納するクラスを簡単に作る機能。インスタンス変数の書き方を簡略化(@??)
from dataclasses import dataclass, field
#決まった選択肢を定義するためのツール(??)
from enum import Enum

#日時、時刻を扱う機能をインポート
from datetime import datetime

#ランダムな値を生成する機能
import random

import hashlib
import json
import re


# ============================================================================
# Spacer Design (Anti-secondary structure, GC 40-60%)
# ============================================================================

class OptimizedSpacerLibrary:
    """
    二次構造を回避し、GC含有率40-60%を維持するspacerライブラリ
    
    設計原則:
    - 回文配列の回避
    - 同一塩基4連続以上の回避
    - 自己相補領域の回避
    - GC含有率40-60%
    """
#各spacer配列を辞書にして定義。
    # 内部spacer（TFBM間）
    INTERNAL_SPACERS = {
        5: [
            'ACTGA',   # GC 40%
            'CAGTA',   # GC 40%
            'GTCAG',   # GC 60%
            'TACGT',   # GC 40%
            'AGTCA',   # GC 40%
        ],
        10: [
            'ACTGAGTCAG',   # GC 50%
            'CAGTACGTAG',   # GC 50%
            'GTCAGATCGA',   # GC 50%
            'TACGTAGTCA',   # GC 40%
            'AGTCATGCAG',   # GC 50%
        ],
    }
    
    # コネクターspacer（TFBMモジュール-Core Promoter間）
    CONNECTOR_SPACERS = {
        4: 'ACTG',      # GC 50%
        7: 'CAGTACG',   # GC 57%
        10: 'ACTGAGTCAG', # GC 50%
    }
    
    # Core Promoter内部spacer
    CORE_SPACERS = {
        5: 'ACTGA',
        6: 'CAGTAC',
        10: 'ACTGAGTCAG',
        15: 'CAGTACGTAGTCAGT',
        18: 'ACTGAGTCAGACTGAGTC',
        20: 'CAGTACGTAGTCAGTACGTA',
        24: 'ACTGAGTCAGACTGAGTCAGACTG',
        27: 'CAGTACGTAGTCAGTACGTAGTCAGTC',
        28: 'ACTGAGTCAGACTGAGTCAGACTGAGTC',
    }
    
    @classmethod
    def get_internal_spacer(cls, length: int, variant_index: int = 0) -> str:
        """内部spacerを取得"""
        if length not in cls.INTERNAL_SPACERS:
            raise ValueError(f"Internal spacer length {length} not supported. Use 5 or 10.")
        spacers = cls.INTERNAL_SPACERS[length]
        return spacers[variant_index % len(spacers)]
    
    @classmethod
    def get_connector_spacer(cls, length: int) -> str:
        """コネクターspacerを取得"""
        if length in cls.CONNECTOR_SPACERS:
            return cls.CONNECTOR_SPACERS[length]
        return cls._generate_custom_spacer(length)
    
    @classmethod
    def get_core_spacer(cls, length: int) -> str:
        """Core Promoter用spacerを取得"""
        if length in cls.CORE_SPACERS:
            return cls.CORE_SPACERS[length]
        return cls._generate_custom_spacer(length)
    
    @classmethod
    def _generate_custom_spacer(cls, length: int) -> str:
        """カスタム長のspacerを生成"""
        base_pattern = 'ACTGAGTCAG'  # GC 50%
        repeats = (length // len(base_pattern)) + 1
        return (base_pattern * repeats)[:length]
    
    @staticmethod
    def validate_spacer(sequence: str) -> Dict[str, any]:
        """spacer配列の品質検証"""
        gc_count = sequence.count('G') + sequence.count('C')
        gc_ratio = gc_count / len(sequence) * 100
        
        has_homopolymer = any(base * 4 in sequence for base in 'ACGT')
        is_palindrome = sequence == sequence[::-1]
        
        complement = str.maketrans('ACGT', 'TGCA')
        rev_comp = sequence.translate(complement)[::-1]
        has_self_comp = any(sequence[i:i+6] in rev_comp for i in range(len(sequence)-5)) if len(sequence) >= 6 else False
        
        return {
            'length': len(sequence),
            'gc_content': round(gc_ratio, 1),
            'has_homopolymer': has_homopolymer,
            'is_palindrome': is_palindrome,
            'has_self_complementary': has_self_comp,
            'quality': 'PASS' if (40 <= gc_ratio <= 60 and not has_homopolymer 
                                  and not is_palindrome and not has_self_comp) else 'WARN'
        }


# ============================================================================
# DNA Barcode Generator (Hamming distance optimized)
# ============================================================================

class DNABarcodeGenerator:
    """
    NGS用DNA barcodeジェネレータ
    
    特徴:
    - 20 bp固定長
    - Hamming distance >= 4
    - GC含量 40-60%
    - ホモポリマー回避
    - 二次構造形成回避
    """
    
    def __init__(self, barcode_length: int = 20, min_hamming_distance: int = 4, seed: int = 42):
        self.barcode_length = barcode_length
        self.min_hamming_distance = min_hamming_distance
        self.generated_barcodes: Set[str] = set()
        self.barcode_map: Dict[str, str] = {}
        random.seed(seed)
    
    def _calculate_hamming_distance(self, seq1: str, seq2: str) -> int:
        return sum(c1 != c2 for c1, c2 in zip(seq1, seq2))
    
    def _check_gc_content(self, sequence: str) -> bool:
        gc_count = sequence.count('G') + sequence.count('C')
        gc_ratio = gc_count / len(sequence)
        return 0.40 <= gc_ratio <= 0.60
    
    def _check_homopolymer(self, sequence: str, max_length: int = 3) -> bool:
        for base in ['A', 'C', 'G', 'T']:
            if base * (max_length + 1) in sequence:
                return False
        return True
    
    def _check_secondary_structure(self, sequence: str) -> bool:
        complement = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
        rev_comp = ''.join(complement[b] for b in reversed(sequence))
        for i in range(len(sequence) - 7):
            if sequence[i:i+8] in rev_comp:
                return False
        return True
    
    def _generate_random_barcode(self) -> str:
        max_attempts = 10000
        for _ in range(max_attempts):
            bases = ['A', 'C', 'G', 'T']
            barcode = ''.join(random.choices(bases, k=self.barcode_length))
            
            if not self._check_gc_content(barcode):
                continue
            if not self._check_homopolymer(barcode):
                continue
            if not self._check_secondary_structure(barcode):
                continue
            
            if all(self._calculate_hamming_distance(barcode, existing) >= self.min_hamming_distance
                   for existing in self.generated_barcodes):
                return barcode
        
        raise ValueError(f"Failed to generate barcode after {max_attempts} attempts")
    
    def generate_barcode(self, promoter_id: str) -> str:
        if promoter_id in self.barcode_map:
            return self.barcode_map[promoter_id]
        
        barcode = self._generate_random_barcode()
        self.generated_barcodes.add(barcode)
        self.barcode_map[promoter_id] = barcode
        return barcode


# ============================================================================
# Constitutive TF Database (~60 TFs)
# ============================================================================

class ConstitutiveTFDatabase:
    """
    Constitutive強発現型TFデータベース
    
    選定基準:
    - Ubiquitous/constitutive発現
    - プロモーター近傍結合傾向
    - 高い転写活性化能
    - JASPAR/HT-SELEXで検証済みモチーフ
    
    除外:
    - Inducible TFs (AP-1, NF-κB, CREB/ATF, HIF, STAT, HSF, p53, etc.)
    """
    
    TF_DATABASE = {
        # ===== SP/KLF Family (GC-box binders) =====
        'SP1': {'consensus': 'GGGCGGG', 'family': 'SP/KLF', 'activity': 1.8, 'jaspar': 'MA0079.5'},
        'SP2': {'consensus': 'GGGCGGG', 'family': 'SP/KLF', 'activity': 1.4, 'jaspar': 'MA0516.2'},
        'SP3': {'consensus': 'GGGCGGG', 'family': 'SP/KLF', 'activity': 1.6, 'jaspar': 'MA0746.2'},
        'SP4': {'consensus': 'GGGCGGG', 'family': 'SP/KLF', 'activity': 1.3, 'jaspar': 'MA0685.1'},
        'KLF2': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.4, 'jaspar': 'MA0513.1'},
        'KLF3': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.3, 'jaspar': 'MA0748.2'},
        'KLF4': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.6, 'jaspar': 'MA0039.4'},
        'KLF5': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.5, 'jaspar': 'MA0599.1'},
        'KLF6': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.3, 'jaspar': 'MA0750.2'},
        'KLF7': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.3, 'jaspar': 'MA0751.1'},
        'KLF9': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.4, 'jaspar': 'MA0517.1'},
        'KLF10': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.3, 'jaspar': 'MA0752.1'},
        'KLF16': {'consensus': 'CACCCAC', 'family': 'SP/KLF', 'activity': 1.3, 'jaspar': 'MA0741.1'},
        
        # ===== ETS Family =====
        'ETS1': {'consensus': 'ACCGGAAGT', 'family': 'ETS', 'activity': 1.5, 'jaspar': 'MA0098.3'},
        'ETS2': {'consensus': 'ACCGGAAGT', 'family': 'ETS', 'activity': 1.4, 'jaspar': 'MA0624.2'},
        'GABPA': {'consensus': 'ACCGGAAGTG', 'family': 'ETS', 'activity': 1.6, 'jaspar': 'MA0062.3'},
        'ELF1': {'consensus': 'CCGGAAGT', 'family': 'ETS', 'activity': 1.6, 'jaspar': 'MA0473.3'},
        'ELF2': {'consensus': 'CCGGAAGT', 'family': 'ETS', 'activity': 1.4, 'jaspar': 'MA0641.1'},
        'ELF4': {'consensus': 'CCGGAAGT', 'family': 'ETS', 'activity': 1.4, 'jaspar': 'MA0641.1'},
        'ETV1': {'consensus': 'ACCGGAAGT', 'family': 'ETS', 'activity': 1.4, 'jaspar': 'MA0761.2'},
        'ETV4': {'consensus': 'ACCGGAAGT', 'family': 'ETS', 'activity': 1.5, 'jaspar': 'MA0764.2'},
        'ETV5': {'consensus': 'ACCGGAAGT', 'family': 'ETS', 'activity': 1.4, 'jaspar': 'MA0765.1'},
        'ETV6': {'consensus': 'ACCGGAAGT', 'family': 'ETS', 'activity': 1.3, 'jaspar': 'MA0645.1'},
        'ERG': {'consensus': 'ACGGAAGT', 'family': 'ETS', 'activity': 1.5, 'jaspar': 'MA0474.3'},
        'FLI1': {'consensus': 'ACCGGAAGT', 'family': 'ETS', 'activity': 1.5, 'jaspar': 'MA0475.2'},
        
        # ===== NF-Y Complex (CCAAT box) =====
        'NFYA': {'consensus': 'GATTGGCTGATTGG', 'family': 'NF-Y', 'activity': 1.7, 'jaspar': 'MA0060.3'},
        'NFYB': {'consensus': 'GATTGGCTGATTGG', 'family': 'NF-Y', 'activity': 1.6, 'jaspar': 'MA0502.2'},
        'NFYC': {'consensus': 'GATTGGCTGATTGG', 'family': 'NF-Y', 'activity': 1.5, 'jaspar': 'MA0501.1'},
        
        # ===== bHLH Family (E-box binders) =====
        'USF1': {'consensus': 'GTCACGTGAC', 'family': 'bHLH', 'activity': 1.6, 'jaspar': 'MA0093.3'},
        'USF2': {'consensus': 'GTCACGTGAC', 'family': 'bHLH', 'activity': 1.6, 'jaspar': 'MA0526.4'},
        'MAX': {'consensus': 'CACGTG', 'family': 'bHLH-LZ', 'activity': 1.5, 'jaspar': 'MA0058.3'},
        'MYC': {'consensus': 'CACGTG', 'family': 'bHLH-LZ', 'activity': 1.9, 'jaspar': 'MA0147.3'},
        'MYCN': {'consensus': 'CACGTG', 'family': 'bHLH-LZ', 'activity': 1.8, 'jaspar': 'MA0104.4'},
        'TFAP4': {'consensus': 'CAGCTG', 'family': 'bHLH', 'activity': 1.4, 'jaspar': 'MA0108.2'},
        'TCF3': {'consensus': 'CAGCTG', 'family': 'bHLH', 'activity': 1.4, 'jaspar': 'MA0522.3'},
        'TCF12': {'consensus': 'CAGCTG', 'family': 'bHLH', 'activity': 1.4, 'jaspar': 'MA0521.1'},
        
        # ===== E2F Family =====
        'E2F1': {'consensus': 'TTTCGCGCGAAA', 'family': 'E2F', 'activity': 1.7, 'jaspar': 'MA0024.3'},
        'E2F2': {'consensus': 'TTTCGCGCGAAA', 'family': 'E2F', 'activity': 1.5, 'jaspar': 'MA0470.2'},
        'E2F3': {'consensus': 'TTTCGCGCGAAA', 'family': 'E2F', 'activity': 1.5, 'jaspar': 'MA0471.2'},
        'E2F4': {'consensus': 'TTTCGCGCGAAA', 'family': 'E2F', 'activity': 1.4, 'jaspar': 'MA0541.2'},
        
        # ===== NRF Family =====
        'NRF1': {'consensus': 'GCGCATGCGC', 'family': 'NRF', 'activity': 1.6, 'jaspar': 'MA0506.2'},
        
        # ===== YY1 Family =====
        'YY1': {'consensus': 'CCGCCATATT', 'family': 'YY1', 'activity': 1.5, 'jaspar': 'MA0095.2'},
        'YY2': {'consensus': 'CCGCCATATT', 'family': 'YY1', 'activity': 1.3, 'jaspar': 'MA0748.1'},
        
        # ===== ZNF/MAZ Family =====
        'ZNF143': {'consensus': 'CCCAGCAG', 'family': 'ZNF', 'activity': 1.7, 'jaspar': 'MA0088.2'},
        'ZNF148': {'consensus': 'GGGGGAGG', 'family': 'ZNF', 'activity': 1.4, 'jaspar': 'MA0752.1'},
        'MAZ': {'consensus': 'GGGAGGG', 'family': 'ZNF', 'activity': 1.6, 'jaspar': 'MA0056.2'},
        'CTCF': {'consensus': 'CCGCGAGGGGGCAG', 'family': 'ZNF', 'activity': 1.4, 'jaspar': 'MA0139.1'},
        
        # ===== TEAD Family =====
        'TEAD1': {'consensus': 'GGAATG', 'family': 'TEAD', 'activity': 1.4, 'jaspar': 'MA0090.3'},
        'TEAD2': {'consensus': 'GGAATG', 'family': 'TEAD', 'activity': 1.4, 'jaspar': 'MA0807.1'},
        'TEAD3': {'consensus': 'GGAATG', 'family': 'TEAD', 'activity': 1.3, 'jaspar': 'MA0808.1'},
        'TEAD4': {'consensus': 'GGAATG', 'family': 'TEAD', 'activity': 1.4, 'jaspar': 'MA0809.2'},
        
        # ===== TFAP2 Family =====
        'TFAP2A': {'consensus': 'GCCGGGGGC', 'family': 'TFAP2', 'activity': 1.5, 'jaspar': 'MA0003.4'},
        'TFAP2C': {'consensus': 'GCCGGGGGC', 'family': 'TFAP2', 'activity': 1.5, 'jaspar': 'MA0524.2'},
        
        # ===== CEBP Family =====
        'CEBPA': {'consensus': 'TTGCGCAA', 'family': 'CEBP', 'activity': 1.5, 'jaspar': 'MA0102.4'},
        'CEBPB': {'consensus': 'TTGCGCAA', 'family': 'CEBP', 'activity': 1.5, 'jaspar': 'MA0466.3'},
        'CEBPG': {'consensus': 'TTGCGCAA', 'family': 'CEBP', 'activity': 1.4, 'jaspar': 'MA0838.1'},
        
        # ===== RUNX Family =====
        'RUNX1': {'consensus': 'TGTGGT', 'family': 'RUNX', 'activity': 1.4, 'jaspar': 'MA0002.2'},
        'RUNX2': {'consensus': 'TGTGGT', 'family': 'RUNX', 'activity': 1.4, 'jaspar': 'MA0511.2'},
    }
    
    @classmethod
    def get_all_tfs(cls) -> pd.DataFrame:
        """全TF情報をDataFrameで返す"""
        data = []
        for tf_name, info in cls.TF_DATABASE.items():
            consensus = info['consensus'].replace('N', 'A')
            data.append({
                'tf_name': tf_name,
                'consensus_sequence': consensus,
                'original_consensus': info['consensus'],
                'length': len(consensus),
                'family': info['family'],
                'activity_score': info['activity'],
                'jaspar_id': info['jaspar'],
            })
        return pd.DataFrame(data)
    
    @classmethod
    def get_tf_families(cls) -> Dict[str, List[str]]:
        """ファミリー別TFリストを返す"""
        families = {}
        for tf_name, info in cls.TF_DATABASE.items():
            family = info['family']
            if family not in families:
                families[family] = []
            families[family].append(tf_name)
        return families
    
    @classmethod
    def get_tf_count(cls) -> int:
        return len(cls.TF_DATABASE)


# ============================================================================
# TFBM Module Generator (5bp / 10bp internal spacer versions)
# ============================================================================

class TFBMModuleGenerator:
    """
    TFBMモジュール生成
    
    特徴:
    - 内部spacer: 5bp / 10bp の2バージョン
    - コピー数: 4x / 6x の2バージョン
    - GC含有率40-60%、二次構造回避設計
    """
    
    def __init__(self):
        self.tf_db = ConstitutiveTFDatabase.get_all_tfs()
        self.copy_numbers = [4, 6]
        self.internal_spacer_lengths = [5, 10]
    
    def generate_module(self, tf_name: str, copy_number: int, spacer_length: int) -> Dict:
        """TFBMモジュールを生成"""
        tf_info = self.tf_db[self.tf_db['tf_name'] == tf_name].iloc[0]
        spacer = OptimizedSpacerLibrary.get_internal_spacer(spacer_length)
        
        parts = []
        for i in range(copy_number):
            parts.append(tf_info['consensus_sequence'])
            if i < copy_number - 1:
                parts.append(spacer)
        
        full_sequence = ''.join(parts)
        
        gc_count = full_sequence.count('G') + full_sequence.count('C')
        gc_content = round(gc_count / len(full_sequence) * 100, 1)
        
        return {
            'tf_name': tf_name,
            'tfbm_sequence': tf_info['consensus_sequence'],
            'copy_number': copy_number,
            'internal_spacer_length': spacer_length,
            'internal_spacer': spacer,
            'full_tfbm_module': full_sequence,
            'module_length': len(full_sequence),
            'tf_family': tf_info['family'],
            'activity_score': tf_info['activity_score'],
            'gc_content': gc_content,
            'jaspar_id': tf_info['jaspar_id'],
        }
    
    def generate_all_modules(self) -> pd.DataFrame:
        """全TFBMモジュールを生成"""
        modules = []
        
        for _, tf in self.tf_db.iterrows():
            for spacer_len in self.internal_spacer_lengths:
                for copy_num in self.copy_numbers:
                    module = self.generate_module(tf['tf_name'], copy_num, spacer_len)
                    modules.append(module)
        
        return pd.DataFrame(modules)


# ============================================================================
# Core Promoter Database (34 types)
# ============================================================================

class CompleteCorePromoterDatabase:
    """34種類の完全Core Promoterデータベース"""
    
    @staticmethod
    def _get_spacer(length: int) -> str:
        return OptimizedSpacerLibrary.get_core_spacer(length)
    
    @classmethod
    def get_all_core_promoters(cls) -> pd.DataFrame:
        """全34種類のCore Promoterを生成"""
        promoters = []
        
        sp5 = cls._get_spacer(5)
        sp6 = cls._get_spacer(6)
        sp10 = cls._get_spacer(10)
        sp15 = cls._get_spacer(15)
        sp18 = cls._get_spacer(18)
        sp20 = cls._get_spacer(20)
        sp24 = cls._get_spacer(24)
        sp27 = cls._get_spacer(27)
        sp28 = cls._get_spacer(28)
        
        # Core Promoter Elements
        bre_opt = 'CGCGCC'
        bre_cons = 'GCCGCC'
        tata_adml = 'TATAAAAG'
        tata_ext = 'TATAAAA'
        tata_cons = 'TATAAA'
        tata_sym = 'TATATATA'
        inr_super = 'CTCAGTCTT'
        inr_adml = 'CTCAGTCT'
        inr_opt = 'CTCAGTCT'
        inr_dual = 'CTCAGTCT'
        inr_weak = 'CTCAGT'
        mte_strong = 'CTAACGGAACGG'
        mte_cons = 'CAAACGGAACGG'
        dpe_ext = 'AGATCCCG'
        dpe_cons = 'AGATC'
        dce_si = 'CTTC'
        dce_sii = 'CTGT'
        dce_siii = 'AGC'
        
        # CpG-like Elements
        sp1_site = 'GGGCGGG'
        ccaat_box = 'GATTGG'
        ets_site = 'GGAAGT'
        ebox_site = 'CACGTG'
        nrf1_site = 'GCGCATGCGC'
        gc_rich_20 = 'GCGCGCGCGCGCGCGCGCGC'
        
        # ========== TATA+ Promoters (8種類) ==========
        seq = bre_opt + sp5 + tata_adml + sp27 + inr_super + sp18 + mte_strong + sp10 + dpe_ext
        promoters.append({
            'core_id': 'CP01', 'group': 'TATA+', 'name': 'Strongest_Full',
            'config': 'BRE_opt + TATA_AdML + INR_super + MTE_strong + DPE_ext',
            'has_tata': True, 'tata_type': 'AdML', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Very High'
        })
        
        seq = tata_adml + sp27 + inr_super + sp18 + mte_strong
        promoters.append({
            'core_id': 'CP02', 'group': 'TATA+', 'name': 'TATA_AdML_INR_super_MTE',
            'config': 'TATA_AdML + INR_super + MTE_strong',
            'has_tata': True, 'tata_type': 'AdML', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = bre_opt + sp5 + tata_adml + sp27 + inr_adml + sp18 + mte_cons
        promoters.append({
            'core_id': 'CP03', 'group': 'TATA+', 'name': 'BRE_TATA_AdML_set',
            'config': 'BRE_opt + TATA_AdML + INR_AdML + MTE_cons',
            'has_tata': True, 'tata_type': 'AdML', 'inr_type': 'AdML',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = tata_ext + sp27 + inr_super + sp18 + mte_strong + sp10 + dpe_ext
        promoters.append({
            'core_id': 'CP04', 'group': 'TATA+', 'name': 'TATA_ext_full',
            'config': 'TATA_ext + INR_super + MTE_strong + DPE_ext',
            'has_tata': True, 'tata_type': 'Extended', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = tata_cons + sp27 + inr_super + sp18 + mte_strong
        promoters.append({
            'core_id': 'CP05', 'group': 'TATA+', 'name': 'TATA_cons_basic',
            'config': 'TATA_cons + INR_super + MTE_strong',
            'has_tata': True, 'tata_type': 'Consensus', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium-High'
        })
        
        seq = tata_sym + sp27 + inr_adml + sp18 + mte_strong
        promoters.append({
            'core_id': 'CP06', 'group': 'TATA+', 'name': 'TATA_sym_variant',
            'config': 'TATA_sym + INR_AdML + MTE_strong',
            'has_tata': True, 'tata_type': 'Symmetric', 'inr_type': 'AdML',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium-High'
        })
        
        seq = bre_cons + sp5 + tata_cons + sp27 + inr_opt + sp18 + mte_cons
        promoters.append({
            'core_id': 'CP07', 'group': 'TATA+', 'name': 'All_consensus',
            'config': 'BRE_cons + TATA_cons + INR_opt + MTE_cons',
            'has_tata': True, 'tata_type': 'Consensus', 'inr_type': 'Optimized',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium'
        })
        
        seq = tata_adml + sp27 + inr_super
        promoters.append({
            'core_id': 'CP08', 'group': 'TATA+', 'name': 'TATA_INR_minimal',
            'config': 'TATA_AdML + INR_super (minimal)',
            'has_tata': True, 'tata_type': 'AdML', 'inr_type': 'Super',
            'has_mte': False, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium'
        })
        
        # ========== TATA- Promoters (12種類) ==========
        seq = inr_super + sp18 + mte_strong + sp10 + dpe_ext
        promoters.append({
            'core_id': 'CP09', 'group': 'TATA-', 'name': 'INR_MTE_DPE_strong',
            'config': 'INR_super + MTE_strong + DPE_ext',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = inr_dual + sp6 + inr_dual + sp18 + mte_strong + sp10 + dpe_ext
        promoters.append({
            'core_id': 'CP10', 'group': 'TATA-', 'name': 'Dual_INR_strong',
            'config': 'Dual_INR + MTE_strong + DPE_ext',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Dual',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = inr_super + sp18 + mte_strong + sp10 + dpe_ext + sp6 + dce_si + dce_sii + dce_siii
        promoters.append({
            'core_id': 'CP11', 'group': 'TATA-', 'name': 'Full_TATA_less',
            'config': 'INR_super + MTE + DPE + DCE_full',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = inr_super + sp18 + mte_strong
        promoters.append({
            'core_id': 'CP12', 'group': 'TATA-', 'name': 'INR_MTE_only',
            'config': 'INR_super + MTE_strong',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium-High'
        })
        
        seq = inr_super + sp28 + dpe_ext
        promoters.append({
            'core_id': 'CP13', 'group': 'TATA-', 'name': 'INR_DPE_only',
            'config': 'INR_super + DPE_ext',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': False, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium'
        })
        
        seq = inr_super + sp6 + inr_dual + sp6 + inr_super
        promoters.append({
            'core_id': 'CP14', 'group': 'TATA-', 'name': 'Triple_INR',
            'config': 'Triple INR (max redundancy)',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Triple',
            'has_mte': False, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium-High'
        })
        
        seq = inr_super + sp18 + mte_cons + sp10 + dpe_cons
        promoters.append({
            'core_id': 'CP15', 'group': 'TATA-', 'name': 'INR_consensus_elements',
            'config': 'INR_super + MTE_cons + DPE_cons',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium'
        })
        
        seq = inr_dual + sp6 + inr_dual + sp18 + mte_strong
        promoters.append({
            'core_id': 'CP16', 'group': 'TATA-', 'name': 'Dual_INR_MTE',
            'config': 'Dual_INR + MTE_strong',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Dual',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium-High'
        })
        
        seq = inr_super
        promoters.append({
            'core_id': 'CP17', 'group': 'TATA-', 'name': 'INR_only_minimal',
            'config': 'INR_super only (ultra-minimal)',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': False, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Low-Medium'
        })
        
        seq = inr_super + sp18 + mte_strong + sp10 + dpe_ext + sp6 + dce_si + dce_sii
        promoters.append({
            'core_id': 'CP18', 'group': 'TATA-', 'name': 'INR_MTE_DPE_DCE_partial',
            'config': 'INR + MTE + DPE + DCE_SI+SII',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = inr_dual + sp6 + inr_dual + sp24 + dpe_ext
        promoters.append({
            'core_id': 'CP19', 'group': 'TATA-', 'name': 'Dual_INR_DPE',
            'config': 'Dual_INR + DPE_ext',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Dual',
            'has_mte': False, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium'
        })
        
        seq = inr_super + sp15 + dce_si + dce_sii + dce_siii
        promoters.append({
            'core_id': 'CP20', 'group': 'TATA-', 'name': 'INR_DCE_only',
            'config': 'INR_super + DCE_full',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': False, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium'
        })
        
        # ========== Literature Promoters (6種類) ==========
        seq = bre_opt + sp5 + tata_adml + sp27 + inr_adml + sp18 + 'CTAACGGAA'
        promoters.append({
            'core_id': 'CP21', 'group': 'Literature', 'name': 'AdML_Complete',
            'config': 'Adenovirus Major Late Promoter',
            'has_tata': True, 'tata_type': 'AdML', 'inr_type': 'AdML',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Very High'
        })
        
        seq = 'TATAAAA' + sp27 + inr_adml
        promoters.append({
            'core_id': 'CP22', 'group': 'Literature', 'name': 'SV40_Early',
            'config': 'SV40 Early Promoter (minimal)',
            'has_tata': True, 'tata_type': 'Extended', 'inr_type': 'Consensus',
            'has_mte': False, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = tata_sym + sp27 + inr_super + sp18 + mte_strong
        promoters.append({
            'core_id': 'CP23', 'group': 'Literature', 'name': 'CMV_Enhanced',
            'config': 'CMV Minimal (enhanced version)',
            'has_tata': True, 'tata_type': 'CMV', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Very High'
        })
        
        seq = tata_cons + sp27 + inr_opt
        promoters.append({
            'core_id': 'CP24', 'group': 'Literature', 'name': 'HSP70_Core',
            'config': 'Heat Shock Protein 70 Core Promoter',
            'has_tata': True, 'tata_type': 'Consensus', 'inr_type': 'Consensus',
            'has_mte': False, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = inr_super + sp18 + mte_strong + sp10 + dpe_ext
        promoters.append({
            'core_id': 'CP25', 'group': 'Literature', 'name': 'Beta_Actin_Core',
            'config': 'β-Actin Core Promoter (TATA-less)',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Super',
            'has_mte': True, 'has_dpe': True, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Very High'
        })
        
        seq = inr_dual + sp6 + inr_dual + sp18 + mte_strong
        promoters.append({
            'core_id': 'CP26', 'group': 'Literature', 'name': 'UBC_Core',
            'config': 'Ubiquitin C Core Promoter (TATA-less)',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Dual',
            'has_mte': True, 'has_dpe': False, 'gc_rich': False,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Very High'
        })
        
        # ========== CpG-like Promoters (8種類) - NEW ==========
        seq = sp1_site + sp6 + sp1_site + sp6 + sp1_site + sp10 + inr_weak
        promoters.append({
            'core_id': 'CpG01', 'group': 'CpG-like', 'name': 'SP1_Cluster',
            'config': '(SP1)×3 + weak_INR (GAPDH-like)',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Weak',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High (Housekeeping)'
        })
        
        seq = ccaat_box + sp10 + sp1_site + sp6 + sp1_site + sp10 + inr_weak
        promoters.append({
            'core_id': 'CpG02', 'group': 'CpG-like', 'name': 'NFY_SP1_Combo',
            'config': 'CCAAT + (SP1)×2 + weak_INR',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Weak',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High (Housekeeping)'
        })
        
        seq = ets_site + sp10 + sp1_site + sp6 + sp1_site + sp10 + inr_weak
        promoters.append({
            'core_id': 'CpG03', 'group': 'CpG-like', 'name': 'ETS_SP1_Combo',
            'config': 'ETS + (SP1)×2 + weak_INR',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Weak',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = sp1_site + sp6 + ccaat_box + sp6 + sp1_site + sp6 + ets_site + sp10 + inr_weak
        promoters.append({
            'core_id': 'CpG04', 'group': 'CpG-like', 'name': 'Triple_Element',
            'config': 'SP1 + CCAAT + SP1 + ETS + weak_INR',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Weak',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Very High'
        })
        
        seq = ebox_site + sp10 + sp1_site + sp6 + sp1_site + sp10 + inr_weak
        promoters.append({
            'core_id': 'CpG05', 'group': 'CpG-like', 'name': 'Ebox_SP1',
            'config': 'E-box + (SP1)×2 + weak_INR',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Weak',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High'
        })
        
        seq = nrf1_site + sp10 + sp1_site + sp6 + sp1_site + sp10 + inr_weak
        promoters.append({
            'core_id': 'CpG06', 'group': 'CpG-like', 'name': 'NRF1_SP1',
            'config': 'NRF1 + (SP1)×2 + weak_INR (Mitochondrial)',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Weak',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High (Mitochondrial)'
        })
        
        seq = gc_rich_20 + sp6 + sp1_site + sp6 + sp1_site + sp10 + inr_weak
        promoters.append({
            'core_id': 'CpG07', 'group': 'CpG-like', 'name': 'Pure_GC_Stretch',
            'config': 'GC-rich(20bp) + (SP1)×2 + weak_INR',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'Weak',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'High (CpG island)'
        })
        
        seq = sp1_site + sp6 + sp1_site + sp6 + ccaat_box + sp6 + sp1_site
        promoters.append({
            'core_id': 'CpG08', 'group': 'CpG-like', 'name': 'Minimal_CpG',
            'config': '(SP1)×2 + CCAAT + SP1 (no INR)',
            'has_tata': False, 'tata_type': 'None', 'inr_type': 'None',
            'has_mte': False, 'has_dpe': False, 'gc_rich': True,
            'sequence': seq, 'length': len(seq), 'expected_activity': 'Medium-High (Dispersed)'
        })
        
        # GC含有率計算
        for p in promoters:
            gc = p['sequence'].count('G') + p['sequence'].count('C')
            p['gc_content'] = round(gc / len(p['sequence']) * 100, 1)
        
        return pd.DataFrame(promoters)


# ============================================================================
# Integrated Promoter Library Generator
# ============================================================================

class IntegratedPromoterLibraryV4:
    """統合人工プロモータライブラリ生成 v4.0"""
    
    def __init__(self, connector_spacer_lengths: List[int] = [4, 7, 10], seed: int = 42):
        self.connector_spacer_lengths = connector_spacer_lengths
        self.tfbm_generator = TFBMModuleGenerator()
        self.barcode_generator = DNABarcodeGenerator(barcode_length=20, min_hamming_distance=4, seed=seed)
    
    def generate_library(self, verbose: bool = True) -> pd.DataFrame:
        """完全な人工プロモータライブラリを生成"""
        
        if verbose:
            print("\n" + "="*80)
            print("🧬 統合人工プロモータライブラリ生成システム v4.0")
            print("="*80)
            print(f"実行時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print("="*80)
        
        if verbose:
            print("\n📦 Step 1: TFBMモジュール生成中...")
        
        tfbm_modules = self.tfbm_generator.generate_all_modules()
        
        if verbose:
            print(f"   ✓ {len(tfbm_modules)} TFBMモジュール生成完了")
            print(f"      - TF数: {tfbm_modules['tf_name'].nunique()}")
            print(f"      - 内部spacer: 5bp / 10bp")
            print(f"      - コピー数: 4x / 6x")
        
        if verbose:
            print("\n🎯 Step 2: Core Promoter準備中...")
        
        core_promoters = CompleteCorePromoterDatabase.get_all_core_promoters()
        
        if verbose:
            print(f"   ✓ {len(core_promoters)} Core Promoter準備完了")
            for group in ['TATA+', 'TATA-', 'Literature', 'CpG-like']:
                count = len(core_promoters[core_promoters['group'] == group])
                print(f"      - {group}: {count}")
        
        if verbose:
            total_combinations = len(tfbm_modules) * len(core_promoters) * len(self.connector_spacer_lengths)
            print(f"\n🔗 Step 3: 統合プロモータ生成中...")
            print(f"   予想プロモータ数: {total_combinations:,}")
        
        promoters = []
        count = 0
        
        for _, tfbm in tfbm_modules.iterrows():
            for _, core in core_promoters.iterrows():
                for connector_len in self.connector_spacer_lengths:
                    
                    promoter_id = (
                        f"{tfbm['tf_name']}_"
                        f"{tfbm['copy_number']}x_"
                        f"int{tfbm['internal_spacer_length']}bp_"
                        f"con{connector_len}bp_"
                        f"{core['core_id']}"
                    )
                    
                    connector = OptimizedSpacerLibrary.get_connector_spacer(connector_len)
                    barcode = self.barcode_generator.generate_barcode(promoter_id)
                    
                    full_sequence = (
                        tfbm['full_tfbm_module'] +
                        connector +
                        core['sequence'] +
                        barcode
                    )
                    
                    gc_count = full_sequence.count('G') + full_sequence.count('C')
                    gc_content = round(gc_count / len(full_sequence) * 100, 1)
                    
                    promoters.append({
                        'promoter_id': promoter_id,
                        'tf_name': tfbm['tf_name'],
                        'tf_family': tfbm['tf_family'],
                        'tfbm_sequence': tfbm['tfbm_sequence'],
                        'copy_number': tfbm['copy_number'],
                        'internal_spacer_length': tfbm['internal_spacer_length'],
                        'internal_spacer': tfbm['internal_spacer'],
                        'tfbm_module': tfbm['full_tfbm_module'],
                        'tfbm_module_length': tfbm['module_length'],
                        'tf_activity_score': tfbm['activity_score'],
                        'jaspar_id': tfbm['jaspar_id'],
                        'connector_spacer_length': connector_len,
                        'connector_spacer': connector,
                        'core_promoter_id': core['core_id'],
                        'core_promoter_name': core['name'],
                        'core_group': core['group'],
                        'core_config': core['config'],
                        'has_tata': core['has_tata'],
                        'tata_type': core['tata_type'],
                        'inr_type': core['inr_type'],
                        'has_mte': core['has_mte'],
                        'has_dpe': core['has_dpe'],
                        'gc_rich': core['gc_rich'],
                        'core_sequence': core['sequence'],
                        'core_length': core['length'],
                        'expected_activity': core['expected_activity'],
                        'barcode': barcode,
                        'full_sequence': full_sequence,
                        'total_length': len(full_sequence),
                        'gc_content': gc_content,
                    })
                    
                    count += 1
            
            if verbose and count % 1000 == 0:
                print(f"   進捗: {count:,}/{total_combinations:,} ({count/total_combinations*100:.1f}%)")
        
        if verbose:
            print(f"   ✓ {count:,} 統合プロモータ生成完了")
        
        df = pd.DataFrame(promoters)
        
        if verbose:
            self._print_statistics(df)
        
        return df
    
    def _print_statistics(self, df: pd.DataFrame):
        """統計情報を表示"""
        print("\n" + "="*80)
        print("📊 ライブラリ統計")
        print("="*80)
        
        print(f"\n【基本情報】")
        print(f"  総プロモータ数: {len(df):,}")
        print(f"  ユニークTF数: {df['tf_name'].nunique()}")
        print(f"  Core Promoter種類: {df['core_promoter_id'].nunique()}")
        
        print(f"\n【TFファミリー分布】")
        for family, count in df.groupby('tf_family').size().sort_values(ascending=False).items():
            print(f"  {family}: {count:,}")
        
        print(f"\n【Core Promoterグループ分布】")
        for group in ['TATA+', 'TATA-', 'Literature', 'CpG-like']:
            count = len(df[df['core_group'] == group])
            print(f"  {group}: {count:,}")
        
        print(f"\n【Spacer設定】")
        print(f"  内部spacer: {df['internal_spacer_length'].unique().tolist()} bp")
        print(f"  コネクターspacer: {df['connector_spacer_length'].unique().tolist()} bp")
        
        print(f"\n【配列統計】")
        print(f"  配列長範囲: {df['total_length'].min()} - {df['total_length'].max()} bp")
        print(f"  平均配列長: {df['total_length'].mean():.1f} bp")
        print(f"  GC含量範囲: {df['gc_content'].min():.1f}% - {df['gc_content'].max():.1f}%")
        print(f"  平均GC含量: {df['gc_content'].mean():.1f}%")
        
        print("="*80 + "\n")
    
    def export_library(self, df: pd.DataFrame, prefix: str = 'synthetic_promoter_library_v4'):
        """ライブラリをエクスポート"""
        
        print("\n📁 ファイルエクスポート中...")
        
        main_file = f"{prefix}_complete.csv"
        df.to_csv(main_file, index=False)
        print(f"  ✅ Complete library: {main_file}")
        
        barcode_file = f"{prefix}_barcode_reference.csv"
        barcode_df = df[['promoter_id', 'barcode', 'tf_name', 'core_group', 'core_promoter_id']].copy()
        barcode_df.to_csv(barcode_file, index=False)
        print(f"  ✅ Barcode reference: {barcode_file}")
        
        tf_file = f"{prefix}_tf_info.csv"
        tf_df = df[['tf_name', 'tf_family', 'tfbm_sequence', 'tf_activity_score', 'jaspar_id']].drop_duplicates()
        tf_df.to_csv(tf_file, index=False)
        print(f"  ✅ TF information: {tf_file}")
        
        core_file = f"{prefix}_core_promoter_info.csv"
        core_cols = ['core_promoter_id', 'core_promoter_name', 'core_group', 'core_config',
                     'has_tata', 'tata_type', 'inr_type', 'has_mte', 'has_dpe', 'gc_rich',
                     'core_sequence', 'core_length', 'expected_activity']
        core_df = df[core_cols].drop_duplicates()
        core_df.to_csv(core_file, index=False)
        print(f"  ✅ Core promoter information: {core_file}")
        
        fasta_file = f"{prefix}_sequences.fasta"
        with open(fasta_file, 'w') as f:
            for _, row in df.iterrows():
                f.write(f">{row['promoter_id']}\n")
                f.write(f"{row['full_sequence']}\n")
        print(f"  ✅ FASTA sequences: {fasta_file}")
        
        summary_file = f"{prefix}_summary.txt"
        with open(summary_file, 'w') as f:
            f.write("="*80 + "\n")
            f.write("Synthetic Promoter Library v4.0 - Summary Report\n")
            f.write("="*80 + "\n\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            f.write("【Library Overview】\n")
            f.write(f"  Total promoters: {len(df):,}\n")
            f.write(f"  Unique TFs: {df['tf_name'].nunique()}\n")
            f.write(f"  Core promoter types: {df['core_promoter_id'].nunique()}\n\n")
            
            f.write("【TF Family Distribution】\n")
            for family, count in df.groupby('tf_family').size().sort_values(ascending=False).items():
                f.write(f"  {family}: {count:,}\n")
            f.write("\n")
            
            f.write("【Core Promoter Group Distribution】\n")
            for group in ['TATA+', 'TATA-', 'Literature', 'CpG-like']:
                count = len(df[df['core_group'] == group])
                f.write(f"  {group}: {count:,}\n")
            f.write("\n")
            
            f.write("【Spacer Settings】\n")
            f.write(f"  Internal spacer lengths: {df['internal_spacer_length'].unique().tolist()} bp\n")
            f.write(f"  Connector spacer lengths: {df['connector_spacer_length'].unique().tolist()} bp\n\n")
            
            f.write("【Sequence Statistics】\n")
            f.write(f"  Length range: {df['total_length'].min()} - {df['total_length'].max()} bp\n")
            f.write(f"  Mean length: {df['total_length'].mean():.1f} bp\n")
            f.write(f"  GC content range: {df['gc_content'].min():.1f}% - {df['gc_content'].max():.1f}%\n")
            f.write(f"  Mean GC content: {df['gc_content'].mean():.1f}%\n\n")
            
            f.write("【TF List】\n")
            for tf in sorted(df['tf_name'].unique()):
                f.write(f"  {tf}\n")
        
        print(f"  ✅ Summary report: {summary_file}")
        print("\n✅ エクスポート完了！\n")
        
        return {
            'main': main_file,
            'barcode': barcode_file,
            'tf': tf_file,
            'core': core_file,
            'fasta': fasta_file,
            'summary': summary_file
        }


# ============================================================================
# NGS Recommendations
# ============================================================================

class NGSRecommendationsV4:
    """NGSシステム推奨"""
    
    @staticmethod
    def calculate_required_reads(library_size: int,
                                 target_reads_per_barcode: int = 100,
                                 sampling_efficiency: float = 0.8,
                                 safety_factor: float = 1.5) -> int:
        base_reads = library_size * target_reads_per_barcode
        adjusted_reads = base_reads / sampling_efficiency * safety_factor
        return int(adjusted_reads)
    
    @staticmethod
    def recommend_platform(library_size: int, target_reads_per_barcode: int = 100):
        required_reads = NGSRecommendationsV4.calculate_required_reads(
            library_size, target_reads_per_barcode
        )
        
        print("\n" + "="*80)
        print("🔬 NGSプラットフォーム推奨")
        print("="*80)
        print(f"\nライブラリサイズ: {library_size:,} プロモータ")
        print(f"目標リード数/バーコード: {target_reads_per_barcode}")
        print(f"必要な総リード数（推定）: {required_reads:,}\n")
        
        platforms = [
            ("Illumina MiSeq v2", 15_000_000, "$1,500-3,000"),
            ("NextSeq 550 Mid Output", 130_000_000, "$3,000-5,000"),
            ("NextSeq 550/2000 High Output", 400_000_000, "$5,000-8,000"),
            ("NovaSeq 6000 S1", 1_600_000_000, "$8,000-12,000"),
        ]
        
        recommended = None
        for name, reads, cost in platforms:
            if required_reads <= reads:
                recommended = (name, reads, cost)
                break
        
        if recommended:
            name, reads, cost = recommended
            print(f"🎯 推奨プラットフォーム: {name}")
            print(f"   - 出力: ~{reads:,} reads")
            print(f"   - コスト: {cost}")
            print(f"   - カバレッジ: ~{reads // library_size}×/barcode (平均)")
        else:
            print("⚠️  大規模ライブラリ: 複数ランまたはNovaSeq S2/S4を検討")
        
        print("\n⚠️  重要な注意:")
        print("   - PhiX spike-in 10-20%を推奨")
        print("   - バーコードのみ: Single-end 75 bp")
        print("   - 全配列検証: Paired-end 2×150 bp")
        print("="*80 + "\n")


# ============================================================================
# Main Execution
# ============================================================================

def main():
    """メイン実行関数"""
    
    library = IntegratedPromoterLibraryV4(
        connector_spacer_lengths=[4, 7, 10],
        seed=42
    )
    
    df = library.generate_library(verbose=True)
    
    files = library.export_library(df, prefix='synthetic_promoter_library_v4')
    
    NGSRecommendationsV4.recommend_platform(
        library_size=len(df),
        target_reads_per_barcode=100
    )
    
    print("\n" + "="*80)
    print("✅ 処理完了！")
    print(f"   {len(df):,} 種類の人工プロモータを生成しました")
    print("="*80 + "\n")
    
    return df


if __name__ == "__main__":
    df = main()



🧬 統合人工プロモータライブラリ生成システム v4.0
実行時刻: 2026-01-25 22:18:08

📦 Step 1: TFBMモジュール生成中...
   ✓ 232 TFBMモジュール生成完了
      - TF数: 58
      - 内部spacer: 5bp / 10bp
      - コピー数: 4x / 6x

🎯 Step 2: Core Promoter準備中...
   ✓ 34 Core Promoter準備完了
      - TATA+: 8
      - TATA-: 12
      - Literature: 6
      - CpG-like: 8

🔗 Step 3: 統合プロモータ生成中...
   予想プロモータ数: 23,664


KeyboardInterrupt: 